<a href="https://colab.research.google.com/github/Gnani1793/DRDO/blob/main/Drdo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import login
login()


In [1]:
# ✅ Step 1: Install required libraries
!pip install -q --upgrade datasets huggingface_hub

# ✅ Step 2: Import necessary libraries
from datasets import load_dataset

# ✅ Step 3: Load the mini-peS2o dataset with authentication
print("📥 Loading mini-peS2o dataset...")
# Changed use_auth_token=True to token=True
dataset = load_dataset("nampdn-ai/mini-peS2o", split="train", token=True)

# ✅ Step 4: Reduce to ~50k samples to keep it ~500MB
dataset = dataset.select(range(50000))

print(f"✅ Loaded {len(dataset)} samples.")

📥 Loading mini-peS2o dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

✅ Loaded 50000 samples.


In [2]:
# ✅ Step 1: Install required libraries (if not already done)
!pip install -q --upgrade transformers accelerate

# ✅ Step 2: Import libraries
from transformers import (
    PhiConfig,
    PhiForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
)
import torch
import numpy as np

# ✅ Step 3: Load tokenizer (from Phi-2)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ✅ Step 4: Tokenize your mini-peS2o dataset
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
        return_special_tokens_mask=True
    )

print("🔧 Tokenizing mini-peS2o dataset...")
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# ✅ Step 5: Define model config from scratch
config = PhiConfig(
    vocab_size=len(tokenizer),   # Important to prevent index errors
    num_hidden_layers=6,         # Adjust based on memory (4–12 layers)
    num_attention_heads=4,
    hidden_size=256,
    intermediate_size=1024,
    max_position_embeddings=128
)

# ✅ Step 6: Initialize model from scratch
model = PhiForCausalLM(config)

# ✅ Step 7: Define data collator for CLM
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # For Causal LM
)

# ✅ Step 8: Define training arguments
training_args = TrainingArguments(
    output_dir="./phi2-mini-pretrained",
    overwrite_output_dir=True,
    num_train_epochs=3,  # Adjust for longer training
    per_device_train_batch_size=8,
    save_steps=500,
    logging_steps=100,
    eval_strategy="no",
    report_to="none",
    fp16=torch.cuda.is_available()
)

# ✅ Step 9: Define a dummy accuracy metric (for illustration)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=-1)
    mask = labels != -100
    correct = (preds == labels) & mask
    accuracy = correct.sum() / mask.sum()
    return {"accuracy": accuracy}

# ✅ Step 10: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# ✅ Step 11: Start training
print("🚀 Pretraining your Phi-style model on mini-peS2o...")
trainer.train()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.5 MB/s eta 0:00:00


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

<ipython-input-2-9ca63d06867b>:76: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🚀 Pretraining your Phi-style model on mini-peS2o...


Step,Training Loss
100,9.806800
200,8.606100
300,7.909300
400,7.528800
500,7.349000
600,7.223800
700,7.100100
800,7.048300
900,6.935600
1000,6.891900


TrainOutput(global_step=18750, training_loss=5.67284552734375, metrics={'train_runtime': 1171.2398, 'train_samples_per_second': 128.069, 'train_steps_per_second': 16.009, 'total_flos': 2034641088000000.0, 'train_loss': 5.67284552734375, 'epoch': 3.0})

In [ ]:
pip install --upgrade transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 88.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers]


In [3]:
import shutil

shutil.make_archive('/content/phi2-mini-pretrained', 'zip', '/content/phi2-mini-pretrained')

'/content/phi2-mini-pretrained.zip'

In [ ]:
from google.colab import files

files.download('/content/phi2-mini-pretrained.zip')